TODO
- embed and save all datasets with models of all sizes for the same seed
- prepare colab compatible notebook loading and training models on the embedded datasets and evaluating on the 

---

In [9]:
import torch

from tqdm import tqdm

from experiments.experiment import Experiment
from experiments.parser import get_parser
from rvae.vizualization.visualization_new import extract_latent_mu
from rvae.utils.paths import SAVED_MODELS_PATH

In [ ]:
model_types = ['VAE', 'RVAE']
latent_dims = [2, 5, 10]

embeddings = {}

parser = get_parser()

for model_type in model_types:
    for latent_dim in latent_dims:
        args = parser.parse_args(
            args=[
                "--model",
                model_type,
                "--dataset",
                "fmnist",
                "--enc_layers",
                "300",
                "300",
                "--dec_layers",
                "300",
                "300",
                "--latent_dim",
                f"{latent_dim}",
                "--num_centers",
                "350",
                "--num_components",
                "1",
                "--device",
                "cpu",
                "--ckpt_path",
                f"{SAVED_MODELS_PATH}/d{latent_dim}/42/{model_type}/{"fmnist_K1epoch100.ckpt" if model_type == "VAE" else "fmnist_epoch100ckpt"}",
            ]
        )
        exp = Experiment(args)
        exp.load_just_model(pretrained_path=args.ckpt_path)
        exp.model._mean_warmup = False
        embeddings[model_type] = embeddings.get(model_type, {})
        embeddings[model_type][latent_dim] = embeddings[model_type].get(latent_dim, {})
        embeddings[model_type][latent_dim]['train'] = {key: value for key, value in zip(["mus", "labels", "idxs"], extract_latent_mu(
            model=exp.model, data_loader=exp.train_loader, model_type=model_type))}
        embeddings[model_type][latent_dim]['test'] = {key: value for key, value in zip(["mus", "labels", "idxs"], extract_latent_mu(
            model=exp.model, data_loader=exp.test_loader, model_type=model_type))}
        
        

DEBUG: self.rvae_save_dir='../saved_models/RVAE/' | os.path.exists(self.rvae_save_dir)=True
DEBUG: self.rvae_save_dir='../saved_models/RVAE/' | os.path.exists(self.rvae_save_dir)=True
DEBUG: self.rvae_save_dir='../saved_models/RVAE/' | os.path.exists(self.rvae_save_dir)=True
DEBUG: self.rvae_save_dir='../saved_models/RVAE/' | os.path.exists(self.rvae_save_dir)=True
DEBUG: self.rvae_save_dir='../saved_models/RVAE/' | os.path.exists(self.rvae_save_dir)=True
DEBUG: self.rvae_save_dir='../saved_models/RVAE/' | os.path.exists(self.rvae_save_dir)=True


In [7]:
embeddings.keys(), embeddings["RVAE"].keys()

KeyError: 'RVAE'

In [ ]:
embeddings["RVAE"][5]["train"]["mus"].shape

torch.Size([60000, 5])

In [ ]:
embeddings["RVAE"][5]["train"]["labels"].shape

torch.Size([60000])

---

In [ ]:
from rvae.utils.paths import RESULTS_PATH

torch.save(embeddings, RESULTS_PATH / "embeddings" / "fmnist_embeddings.pt")

In [ ]:
RESULTS_PATH / "embeddings" / "fmnist_embeddings.pt"

PosixPath('/home/prz/PROJECTS/rvae_reproduction/results/embeddings/fmnist_embeddings.pt')

In [ ]:
embeddings = torch.load(RESULTS_PATH / "embeddings" / "fmnist_embeddings.pt")

/tmp/ipykernel_72343/2297167075.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load(RESULTS_PATH / "embeddings" / "fmnist_embeddings.pt")
